In [31]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken

### Getting the data

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Indexing the data

In [4]:
es_client = Elasticsearch('http://localhost:9200') 
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [5]:
# index documents by elastic search
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Searching
Now let's search in our index.\
We will execute a query "How do execute a command on a Kubernetes pod?".\
Use only question and text fields and give question a boost of 4, and use "type": "best_fields".\
What's the score for the top ranking result?

In [6]:
query = "How do execute a command on a Kubernetes pod?"

In [11]:
search_query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
            
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [12]:
response['hits']['hits'][0]['_score']

44.50556

### Filtering
Now ask a different question: "How do copy a file to a Docker container?".\
This time we are only interested in questions from machine-learning-zoomcamp.\
Return 3 results. What's the 3rd question returned by the search engine?

In [13]:
query = "How do copy a file to a Docker container?"

In [14]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

# take only text from response without metadata info
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [18]:
result_docs[2]

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

### Building a prompt
Now we're ready to build a prompt to send to an LLM.\
Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks

In [22]:
context_template = ''
    
for doc in result_docs:
    context_template = context_template + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

In [23]:
print(context_template)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

In [24]:
query = "How do I execute a command in a running docker container?"

In [25]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [26]:
prompt = prompt_template.format(question=query, context=context_template).strip()

In [28]:
len(prompt)

1462

### Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.\
Let's calculate the number of tokens in our query:

In [37]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [38]:
len(encoding.encode(prompt))

322

In [39]:
# decode back
encoding.decode_single_token_bytes(63842)

b"You're"

### generating the answer

### calculating the costs
Suppose that on average per request we send 150 tokens and receive back 250 tokens.\
How much will it cost to run 1000 requests?

On June 17 2024, the prices for gpt4o are:\
Input: 0.005 - 1K tokens \
Output: 0.015 - 1K tokens \
You can redo the calculations with the values you got in Q6 and Q7.

In [45]:
150 * 0.005 + 250 * 0.015

4.5

for gpt4o on 2025-06-04:\
Input: 0.004 - 1K tokens\
Output: 0.016 - 1K tokens

In [50]:
150 * 0.004 + 250 * 0.016

4.6

In [49]:
1462 * 0.004 + 250 * 0.016

9.847999999999999